# Download Libs

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.5 MB/s 
     |████████████████████████████████| 457 kB 40.3 MB/s 
     |████████████████████████████████| 181 kB 39.5 MB/s 
     |████████████████████████████████| 10.1 MB 28.2 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 660 kB 49.5 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

# Restart Runtime

In [ ]:
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 25 kB/s 
     |████████████████████████████████| 51 kB 152 kB/s 
     |████████████████████████████████| 3.8 MB 10.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.8 MB/s 
     |████████████████████████████████| 880 kB 50.7 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 52.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b3f9e05eb70798e5a3f77d9373379c30f1500e257a4e2d7f2365792179a3a806
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-

In [ ]:
# pip install spacy-transformers -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


# Download Data

In [ ]:
from google.colab import files
files.upload() #enviar o arquivo kaggle.json

#antes de importar o dataset nós iremos provisionar o local de armazenamento
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json


!rm ./data/*
!kaggle competitions download -c sentiment-analysis-on-movie-reviews
!unzip ./sentiment-analysis-on-movie-reviews.zip -d ./data/
!rm sentiment-analysis-on-movie-reviews.zip
!unzip ./data/test.tsv.zip -d ./data/
!unzip ./data/train.tsv.zip -d ./data/
!rm ./data/train.tsv.zip
!rm ./data/test.tsv.zip

Saving kaggle.json to kaggle.json
rm: cannot remove './data/*': No such file or directory
  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 126MB/s]
Archive:  ./sentiment-analysis-on-movie-reviews.zip
  inflating: ./data/sampleSubmission.csv  
  inflating: ./data/test.tsv.zip     
  inflating: ./data/train.tsv.zip    
Archive:  ./data/test.tsv.zip
  inflating: ./data/test.tsv         
Archive:  ./data/train.tsv.zip
  inflating: ./data/train.tsv        


# Read Data

In [ ]:
import pandas as pd

test = pd.read_csv('./data/test.tsv', sep='\t')
train = pd.read_csv('./data/train.tsv', sep='\t')
# sample_submission = pd.read_csv('./data/sampleSubmission.csv', sep='\t')

In [ ]:
test

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [ ]:
train = train.drop_duplicates(subset = "SentenceId")

In [ ]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


# Tokenization

In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [ ]:
from nltk.corpus import stopwords
stopwords = [word for word in stopwords.words('english')] + [word for word in nlp.Defaults.stop_words]
stopwords = [word.lower() for word in stopwords]

def tokenize(text):
	doc = nlp(text)
	tokens = [token.orth_.lower() for token in doc if token.is_alpha and (token.orth_.lower() not in stopwords)]
	text_pre = None
	for token in tokens:
		if text_pre is None:
			text_pre = token
		else:
			text_pre = text_pre + ' ' + token
	return text_pre

In [ ]:
def lemmatize(text):
	doc = nlp(text)
	tokens = [token.lemma_ for token in doc]
	text_lemma = None
	for token in tokens:
		if text_lemma is None:
			text_lemma = token
		else:
			text_lemma = text_lemma + ' ' + token
	return text_lemma

In [ ]:
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
def stematize(text):
	doc = nlp(text)	
	tokens = [stemmer.stem(token.orth_) for token in doc]
	text_stema = None
	for token in tokens:
		if text_stema is None:
			text_stema = token
		else:
			text_stema = text_stema + ' ' + token
	return text_stema

In [ ]:
def pre_process(text):
	if text is None:
		return text
	else: 
		return stematize(lemmatize(text))

In [ ]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


In [ ]:
train['Phrase'] = train['Phrase'].map(lambda text: pre_process(text))

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a seri of escapad demonstr the adag that what ...,1
63,64,2,"thi quiet , introspect and entertain independ ...",4
81,82,3,"even fan of ismail merchant 's work , I suspec...",1
116,117,4,a posit thrill combin of ethnographi and all t...,3
156,157,5,aggress self - glorif and a manipul whitewash .,1
...,...,...,...,...
155984,155985,8540,... either you be will to go with thi claustro...,2
155997,155998,8541,"despit these annoy , the capabl clayburgh and ...",2
156021,156022,8542,-lrb- tri -rrb- to parodi a genr that be alrea...,1
156031,156032,8543,the movi 's downfal be to substitut plot for p...,1


In [ ]:
train.to_csv("./train.csv")

In [ ]:
for i in range(1, 100, 1):
    print(pre_process(train.Phrase[i]))

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


seri escapad demonstr adag good goos
seri
None
seri
escapad demonstr adag good goos
None
escapad demonstr adag good goos
escapad
demonstr adag good goos
demonstr adag
demonstr
adag
None
adag
good goos
None
good goos
None
good goos
None
good goos
good
goos
None
goos
goos
good gander occasion amus amount stori
good gander occasion amus amount stori
None
None
good gander occasion amus amount stori
gander occasion amus amount stori
gander occasion amus amount stori
gander
gander
gander
None
occasion amus amount stori
None
None
None
None
occasion amus amount stori
occasion
amus amount stori
amus
amount stori
None
amount stori
None
amount stori
amount stori
amount stori
amount
stori
None
stori
None
stori
stori
stori
None
quiet introspect entertain independ worth seek
quiet introspect entertain independ
None
quiet introspect entertain independ
quiet introspect entertain
quiet
introspect entertain
introspect entertain
introspect
introspect
None
entertain
independ
worth seek
worth seek
worth
wo